# Introduction

This notebook conducts an exploratory data analysis of the dataset provided for my work trial project. 

At the end of this notebook, we will have:

* Imported both the customer application and the loan performance data
* Merged the loan performance data into the application data, using assumptions explained below
* Generated an .html report for each variable using Pandas Profiling
* Constructed a dynamic data dictionary that will be carried through the project

Merging the loan performance records with the customer application records posed several challenges. 
* First, the two datasets do not share a common key, leaving no natural way to merge the two. To address this problem, I built a customer_id in the loan performance table using the provided idLoan. Assumptions about the definition of customer_id are verified below. 
* Second, coverage is imperfect on both sides. 13 customers have no corresponding loan record, and 688 loans have no customer. I merge and keep only records that exist in both datasets, and then explore the implications of the failed merge. 
    * Importantly, loans that had no customer data are 46% less likely to be 'bad' loans than matched records. This indicates that the provided sample is not representative of the overall population, and that we might face challenges when generalizing out of sample. 
    * This adds to other concerns about the sampling strategy used to obtain this dataset. For example, this dataset only includes customers who were approved to receive a loan. Any and all rejected applications are hidden from view, making this dataset markedly different from the overall pool of applicants to which our algorithm will be asked to generalize. 
* Third, there appear to be some customers who have multiple loan records. I assume this is by design and conduct a 1:m merge.

The Data Dictionary constructed in this notebook will be essential for automating the model pipeline going forwards. As currently constructed, the Data Dictionary is a DataFrame with a row for each variable reporting three pieces of information:
* Data Type
* Categorical Variable Flag
* % Coverage

This exploratory analysis paves the way for future analysis including feature engineering, dimensionality reduction, model construction, performance evaluation, and fair lending analysis. 

In [1]:
import sys
import numpy as np
import pandas as pd
import openpyxl
from datetime import datetime
from numpy import datetime64

from utils import bad_rate_by_category

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 50)

## Import Data

In [2]:
# Import data from provided Excel file

application_data = pd.read_excel(io = 'input_data/Homework - Data Scientist.xlsx',
                     sheet_name = 'Application Data',
                     header = 0,
                     engine='openpyxl')

performance_data = pd.read_excel(io = 'input_data/Homework - Data Scientist.xlsx',
                             sheet_name = 'Loan Performance',
                             header = 0,
                             engine='openpyxl')

C:\Users\aidan\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Warning message caused by unknown extension to .xlsx file, such as filtering or conditional formatting. Rather than alter the provided file or suppress the warning, I will continue by using the data as it was imported. 

## Simple Data Exploration

In [3]:
# Application Data columns and data types
print(application_data.dtypes)

customer_id                            object
amount_requested                        int64
birth_date                     datetime64[ns]
status                                 object
residence_rent_or_own                    bool
monthly_rent_amount                     int64
bank_account_direct_deposit              bool
application_when               datetime64[ns]
loan_duration                           int64
payment_ach                              bool
num_payments                            int64
payment_amount                        float64
amount_approved                         int64
duration_approved                       int64
payment_amount_approved               float64
address_zip                             int64
email                                  object
bank_routing_number                     int64
email_duration                         object
residence_duration                     object
bank_account_duration                  object
payment_frequency                 

In [4]:
# Application Data itself
application_data.head()

,customer_id,amount_requested,birth_date,status,residence_rent_or_own,monthly_rent_amount,bank_account_direct_deposit,application_when,loan_duration,payment_ach,num_payments,payment_amount,amount_approved,duration_approved,payment_amount_approved,address_zip,email,bank_routing_number,email_duration,residence_duration,bank_account_duration,payment_frequency,home_phone_type,other_phone_type,how_use_money,monthly_income_amount,raw_l2c_score,raw_FICO_telecom,raw_FICO_retail,raw_FICO_bank_card,raw_FICO_money
0,9ece67d6c5,500,1991-08-21,Approved,True,0,True,2010-10-16 14:06:05,3,True,6,138.48,500,3,NaN,84118,WanitaGVinyard@gmail.com,124001545,1 year or more,6 months or less,6 months or less,Semi-monthly,Mobile,NaN,Other,1560,614,574,600,656,561
1,5c2c402094,500,1990-07-24,Approved,True,0,False,2010-10-17 13:01:53,6,True,13,77.39,500,6,NaN,84062,NatashaTTucker@yahoo.com,124000054,1 year or more,3+ years,3+ years,Bi-weekly,Mobile,NaN,Other,900,708,501,550,651,563
2,e6254cad30,400,1980-09-13,Approved,True,620,True,2010-10-17 19:52:34,6,True,13,61.91,400,6,NaN,84119,AnneDCisneros@hotmail.com,124001545,1 year or more,7-12 months,1-2 years,Bi-weekly,Mobile,NaN,Other,1434,687,522,561,661,598
3,49fb42f51d,500,1976-09-10,Approved,False,785,False,2010-10-18 07:05:00,4,True,8,108.48,500,4,NaN,84405,CatalinaRRoderick@hotmail.com,124002971,1 year or more,3+ years,7-12 months,Bi-weekly,Mobile,NaN,Other,1600,616,560,552,634,591
4,195fbe5739,300,1979-12-01,Approved,True,700,True,2010-10-19 10:58:40,4,True,8,65.09,300,4,NaN,84404,PhillipMSinn@aol.com,124002971,1 year or more,3+ years,6 months or less,Bi-weekly,Mobile,Home,Other,1360,681,603,654,659,636


In [5]:
# Performance Data columns and data types
print(performance_data.dtypes)

idLoan     object
flgGood    object
dtype: object


In [6]:
# Performance Data itself
performance_data.head()

,idLoan,flgGood
0,0DBA6B011D-01,Bad
1,96ACBBA7ED-01,Bad
2,289421D82A-01,Bad
3,9ECE67D6C5-01,Good
4,5C2C402094-01,Bad


## One Hot Encoding for Performance

In [7]:
# Generate columns 'bad' and 'good' with dummies for 'flgGood'
performance_data[['bad', 'good']] = pd.get_dummies(performance_data['flgGood'])

## Merge Application Data and Performance Data

Two questions arise when merging the data. 

First, the datasets do not share a key for merging on. The closest thing to a common key is the 'customer_id' in application_data paired to the 'idLoan' in performance data. But 'idLoan's are different from customer_ids in a key way: they have appended to the end of the customer_id an additional '-01' or '-02'. 

Presumably, each loan record refers to a loan held by a customer, and each customer can have one or more loans. We will want to train and test our model on all loans held by all students, conserving information and increasing the potential performance of the model. To that end, we validate these assumptions and construct a 'customer_id' in the performance data. 

Second, the 1:m relationship between application and performance data does not have 100% coverage on either side. We must show which data cannot be matched between the two sets, and consider the implications of that non-matching. 

The code below verifies this diagnosis and implements the recommended solution. 

In [8]:
# Generate customer_id in performance_data
# Assumption: Customer ID is lowercase Loan ID without the last three chars
performance_data['customer_id'] = performance_data['idLoan']
performance_data['customer_id'] = performance_data['customer_id'].str.lower()
performance_data['customer_id'] = performance_data['customer_id'].str.slice(start = 0, stop = -3)

# How many customers have multiple loans in the data?
repeat_customers = performance_data['customer_id'].duplicated().sum()
print(f"Number of customers with multiple loans in the data: {repeat_customers}")

Number of customers with multiple loans in the data: 16


In [9]:
# Merge Coverage: How many observations can be merged?
# Use Python set object to evaluate
s1 = set(application_data['customer_id'])
s2 = set(performance_data['customer_id'])
s1.intersection(s2)

match_both = s1.intersection(s2)
match_application = s1 - s2
match_performance = s2 - s1

print(f"Matched in both datasets: {len(match_both)}")
print(f"Unmatched from application dataset: {len(match_application)}")
print(f"Unmatched from performance dataset: {len(match_performance)}")
print()

# TODO: Make one of those cool conversion funnel charts
# Applications      Performance
# 650               650 (Match)
# 13                0 (No performance)
# 0                 688 (No application)


# Merge performance_data into application_data
performance_subset = ['customer_id', 'idLoan', 'bad']

data = application_data.merge(performance_data[performance_subset], 
                               how = 'inner', 
                               on = 'customer_id')

print(f"Therefore, after merging, we expect {len(match_both) + repeat_customers} records.")
print(f"Merged data contains {data.shape[0]} records.")

Matched in both datasets: 634
Unmatched from application dataset: 13
Unmatched from performance dataset: 635

Therefore, after merging, we expect 650 records.
Merged data contains 650 records.


## Implications of Unmerged Data

Loans which could be matched to customer applications have **more than twice as likely to be bad** as loans that went unmatched to a customer application. 

This indicates that our data is not randomly sampled from the overall population which would limit the model's out-of-sample generalized performance. It is possible that default rates will usually be lower on out-of-sample data because the only such data we have access to shows lower default rates. 

One crucial followup for this project would be determining the filtering criteria for our two data samples, assessing the potential impact on model performance, and possibly gather more data under different criteria. 



In [10]:
# Analyzing the unmerged loan performance records
performance_data = performance_data.merge(application_data['customer_id'],
                                           how = 'left',
                                           on = 'customer_id',
                                           indicator = True)

performance_gb = performance_data.groupby('_merge').agg({'bad': ['sum','count']})
performance_gb = performance_gb.loc[['left_only', 'both']]
performance_gb['bad_rate'] = performance_gb[('bad', 'sum')] / performance_gb[('bad', 'count')]
performance_gb.index = ['No Applicant Matched', 'Applicant Matched']
performance_gb[['bad_rate']]

,bad_rate
,
No Applicant Matched,0.256693
Applicant Matched,0.543077


## Building the Data Dictionary

In [11]:
# Build data dictionary
data_dict = pd.DataFrame()
data_dict['variable'] = data.columns
data_dict['var_dtype'] = data.dtypes.values

In [12]:
# Hand annotation of variable categories
# Manual first-pass before automated dimensionality reduction 

category_dict = {
    'customer_id': 'performance', 
    'amount_requested': 'terms',
    'birth_date': 'protected', # Age is a protected class
    'status': 'terms', # All are approved -- missing the rejected population
    'residence_rent_or_own': 'personal_finance',
    'monthly_rent_amount': 'personal_finance',
    'bank_account_direct_deposit': 'personal_finance',
    'application_when': 'application', # Detect seasonality in submissions / defaults? 
    'loan_duration': 'application',  
    'payment_ach': 'personal_finance', # All but one are true -- not useful
    'num_payments': 'terms', 
    'payment_amount': 'terms',
    'amount_approved': 'terms',
    'duration_approved': 'terms', # Might affect default rates
    'payment_amount_approved': 'terms',
    'address_zip': 'protected', # DI Analysis: ZIP -> Census data -> Race
    'email': 'protected', # DI Analysis: Email -> Last name -> Census data -> Race
    'bank_routing_number': 'protected', # Extract bank info
    'email_duration': 'other_info', # DI correlation with age
    'residence_duration': 'other_info', # DI correlation with age
    'bank_account_duration': 'personal_finance', # DI correlation with age
    'payment_frequency': 'personal_finance', 
    'home_phone_type': 'other_info', 
    'other_phone_type': 'other_info', 
    'how_use_money': 'personal_finance', # Allow for different produces
    'monthly_income_amount': 'personal_finance',
    'raw_l2c_score': 'credit_score',
    'raw_FICO_telecom': 'credit_score',
    'raw_FICO_retail': 'credit_score',
    'raw_FICO_bank_card': 'credit_score',
    'raw_FICO_money': 'credit_score',
    'idLoan': 'performance', # Separate loans are separate records
    'flgGood': 'performance',
    'bad': 'performance', # Focus on 'bad' outcome
    'good': 'performance'
}

data_dict['eda_category'] = data_dict.apply(lambda x: category_dict[x.variable], axis=1)

In [13]:
# Categorical vs. Continuous tagging
"""
Whether a variable is categorical or continuous is one of the
most important facts to know about it throughout the ML pipeline. 
It affects your data cleaning (one-hot encoding), your dimensionality
reduction (ANOVA vs Chi Square), and modeling (tagging categoricals).

A more thorough analysis would indicate binary vs nominal vs ordinal. 

"""

# Convert np.NaN to None for non-numeric columns
data = data.where(pd.notnull(data), None)


# Convert types as desired
data = data.astype({
    'customer_id': str,
    'status': str,
    'address_zip': str,
    'email': str,
    'bank_routing_number': str,
    'email_duration': str,
    'residence_duration': str,
    'bank_account_duration': str,
    'payment_frequency': str,
    'home_phone_type': str,
    'other_phone_type': str,
    'how_use_money': str,
    'idLoan': str
})

# Update dictionary with new types
data_dict['var_dtype'] = data.dtypes.values

# Categorization by type
type_categorization = {
    'object': 1,
    'bool': 1,
    'int64': 0,
    'uint8': 0,
    'float64': 0,
    'datetime64[ns]': 1
}

data_dict['categorical'] = data_dict.apply(lambda x: type_categorization[str(x.var_dtype)], axis=1)

In [14]:
# Coverage for each variable
data_dict['coverage'] = data.count().values / data.shape[0]

In [15]:
data_dict.head()

,variable,var_dtype,eda_category,categorical,coverage
0,customer_id,object,performance,1,1.0
1,amount_requested,int64,terms,0,1.0
2,birth_date,datetime64[ns],protected,1,1.0
3,status,object,terms,1,1.0
4,residence_rent_or_own,bool,personal_finance,1,1.0


## Pandas Profiling

In [16]:
from pandas_profiling import ProfileReport

profile = ProfileReport(data, minimal=True)

profile.to_file('final_products/eda_profile.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Export Data

In [17]:
data.to_pickle('output_data/00_data.pkl')
data_dict.to_pickle('output_data/00_data_dict.pkl')